In [1]:
# Import required packages
import torch
import os
import numpy as np

from utils.solver import Solver, DEFAULT_SOLVER_PARAM_M7
from utils.robot import get_robot, sample_P_path, sample_J_traj
from utils.utils import load_numpy, save_numpy, nearest_neighbor_F

In [2]:
robot = get_robot()
solver = Solver(robot=robot, solver_param=DEFAULT_SOLVER_PARAM_M7)

In [ ]:
num_poses = 100
num_sols = 1000

In [5]:
# solver.latent = torch.zeros(1, 7).cuda()
# solver.shirnk_ratio = 0.31
solver.random_evaluation(num_poses, num_sols, return_time=True)

In [ ]:
# solver.latent = torch.zeros(1, 7).cuda()
solver.shirnk_ratio = 0.31
solver.random_evaluation(num_poses, num_sols, return_time=True)

In [50]:
Path_dir = sample_P_path(load_time='', num_steps=20)

mkdir /home/luca/ikpflow/data/panda/trajectory/0925141003
/home/luca/ikpflow/data/panda/trajectory/0925141003/ load successfully.


In [51]:
def path_following(
    solver,
    Path_dir: str,
    num_traj: int = 3,
) -> None:
    """
    path following generation for our method
    
    Parameters
    ----------
    robot : _type_
        robotic arm
    Path_dir : str
        path to ee Path, generated by sample_P_path
    model : _type_
        flow, iflow, or nflow
    knn : _type_
        knn of P_train
    F : _type_
        F_train
    num_traj : int, optional
        the number of generated joint trajectory samples, by default 3
    """

    def load_and_plot(exp_traj_path: str):
        if os.path.exists(path=exp_traj_path):
            qs = load_numpy(exp_traj_path)
            robot.plot(qs=qs)
        else:
            print(f"{exp_traj_path} does not exist !")

    Path = load_numpy(file_path=Path_dir + "ee_traj.npy")
    # Path = Path[:, :3]

    ref_F = nearest_neighbor_F(knn, np.atleast_2d(Path[0]), F, n_neighbors=40_0000) # knn
    # ref_F = F
    # ref_F = rand_F(Path[0], F)
    
    exp_path = lambda idx: Path_dir + f"exp_{idx}.npy"
    
    # rand_idxs = np.random.randint(low=0, high=len(ref_F), size=num_traj)
    rand_idxs = [0, 0, 0]

    # rand_idxs = list(range(num_traj))
    for i, rand in enumerate(rand_idxs):
        # ref_F = rand_F(Path[0], F)
        
        # df, qs = sample_J_traj(Path, ref_F[rand], model, robot) 
        df, qs = sample_J_traj(Path, ref_F[rand], model, robot) 
        print("="*6 + str(rand) + f"=({ref_F[rand]})" + "="*6)
        print(df.describe())
        save_numpy(file_path=exp_path(i), arr=qs)
        load_and_plot(exp_traj_path=exp_path(i))

In [53]:
path_following(robot=panda, Path_dir=Path_dir, model=iflow, knn=knn, F=F, num_traj=3)

======0=([-0.01014882 -0.14512376  0.20029698  0.57698549])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.003456      60.764851
std           0.002047      31.326494
min           0.000504      19.083036
25%           0.002279      43.887614
50%           0.002992      49.754499
75%           0.004504      71.584299
max           0.009133     148.332687
======0=([-0.01014882 -0.14512376  0.20029698  0.57698549])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.003294      53.404072
std           0.001815      27.112625
min           0.000411      20.482630
25%           0.001831      34.068739
50%           0.003255      48.873785
75%           0.004138      62.836220
max           0.008593     136.959579
======0=([-0.01014882 -0.14512376  0.20029698  0.57698549])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.003858      56.276626
std 

In [ ]:
path_following(robot=panda, Path_dir=Path_dir, model=nflow, knn=knn, F=F)

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
target_poses = load_numpy(file_path=traj_dir + 'ee_traj.npy')
# traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
mean_position_errors = np.zeros((num_ikflow_trails,))
mean_ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, position_errors, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = eval_J_traj(robot=panda, J_traj=solutions.to('cpu'), P_path=target_poses, position_errors=position_errors)
    mean_position_errors[ikflow_i], mean_ang_errs[ikflow_i]  = df.mean().values
data['ikflow_l2'] = mean_position_errors
data['ikflow_ang'] = mean_ang_errs    


In [ ]:
df = pd.DataFrame(mean_position_errors)
df.describe()

In [ ]:
df = pd.DataFrame(mean_ang_errs)
df.describe()